# Aprendizagem por Reforço com Q-Learning

### Entendendo o ambiente

In [1]:
! pip install gym
! pip install gym==0.17.3

In [2]:
import gym
import random
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

'0.17.3'

In [4]:
env = gym.make('Taxi-v3')

In [5]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [7]:
# 0 = south, 1 = north, 2 = east, 3 = west, 4 = pickup, 5 = dropoff
print(env.action_space)

Discrete(6)


In [8]:
# 4 destinos 5 * 5 * 5 * 4
print(env.observation_space)

Discrete(500)


In [10]:
len(env.P)

500

In [12]:
env.P[484]

{0: [(1.0, 484, -1, False)],
 1: [(1.0, 384, -1, False)],
 2: [(1.0, 484, -1, False)],
 3: [(1.0, 464, -1, False)],
 4: [(1.0, 484, -10, False)],
 5: [(1.0, 484, -10, False)]}

### Treinamento

In [13]:
random.uniform(0,1)

0.3840006246688329

In [19]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table.shape

(500, 6)

In [20]:
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [21]:
%%time
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1

for i in range(100000):
    estado = env.reset()

    penalidades, recompensa = 0, 0
    done = False
    while not done:
      # Exploração
        if random.uniform(0, 1) < epsilon:
          action = env.action_space.sample()
      # Exploitation
        else:
          action = np.argmax(q_table[estado])

        proximo_estado, recompensa, done, info = env.step(action)

        q_antigo = q_table[estado, action]
        proximo_maximo = np.max(q_table[proximo_estado])
        q_novo = (1 - alpha) * q_antigo + alpha * (recompensa + gamma * proximo_maximo)
        q_table[estado, action] = q_novo
        if recompensa == -10:
          penalidades += 1

        estado = proximo_estado
    if i % 100 == 0:
      clear_output(wait=True)
      print(f'Episódio: {i}')

print("Treinamento concluido")

Episódio: 99900
Treinamento concluido
CPU times: user 1min, sys: 5.78 s, total: 1min 6s
Wall time: 1min 8s


In [26]:
q_table[482]

array([-2.47461784, -2.47061344, -2.47192661, -2.47061344, -8.46288046,
       -8.11646336])

In [24]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [25]:
env.encode(4, 4, 0, 2)

482

### Avaliação

In [27]:
total_penalidades = 0
episodios = 50
frames = []

for _ in range(episodios):
  estado = env.reset()
  penalidades, recompensa = 0, 0
  done = False
  while not done:
    action = np.argmax(q_table[estado])
    estado, recompensa, done, info = env.step(action)

    if recompensa == -10:
      penalidades += 1

    frames.append({
        'frame': env.render(mode='ansi'),
        'state': estado,
        'action': action,
        'reward': recompensa
    })
  total_penalidades += penalidades

print(f"Episodio: {episodios}")
print(f"Penalidades: {total_penalidades}")

Episodio: 50
Penalidades: 0


In [29]:
frames[0]

{'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n|\x1b \x1b| : | : |\n|Y| : |B: |\n+---------+\n  (North)\n',
 'state': 301,
 'action': 1,
 'reward': -1}

In [32]:
from time import sleep
for frame in frames:
  clear_output(wait=True)
  print(frame['frame'])
  print(f"Estado: {frame['state']}")
  print(f"Ação: {frame['action']}")
  print(f"Recompensa: {frame['reward']}")
  sleep(0.3)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

Estado: 234
Ação: 0
Recompensa: -1


KeyboardInterrupt: 